In [51]:
import json, folium
import numpy as np
import pandas as pd

### 서울시 공원
- 자치구별 공원 면적 단계 구분도

In [52]:
s_park = pd.read_csv('data/서울공원요약.csv')
s_park.head()

,공원명,면적,주소,경도,위도,분류,크기
0,남산도시자연공원,2896887,서울특별시 중구 삼일대로 231(예장동),126.990377,37.550140,대형,15
1,길동생태공원,80683,서울특별시 강동구 천호대로 1291(길동생태공원),127.154779,37.540394,소형,3
2,서울대공원,9132690,경기도 과천시 대공원광장로 102,127.019846,37.426449,대형,15
3,서울숲,480994,서울특별시 성동구 뚝섬로 273 (성수동1가),127.041798,37.543072,중형,7
4,월드컵공원,2284085,서울특별시 마포구 하늘공원로 84(월드컵공원),126.878907,37.571805,대형,15


In [53]:
s_park = s_park.drop(s_park[s_park.주소.str.contains('경기도')].index)
gu_list = []
for line in s_park.주소:
    if len(line.split()[0]) == 5:
        gu = line.split()[1]
    else:
        gu = line.split()[0]
    gu_list.append(gu)

s_park['주소'] = gu_list
s_park = s_park[['면적', '주소']]
s_park = s_park.groupby('주소')['면적'].sum()
s = pd.DataFrame(s_park)
s.head()

,면적
주소,
강남구,3626398
강동구,1526387
강북구,2047054
강서구,2194152
관악구,10431453


In [54]:
filename = 'data/seoul_geo_simple.json'
geo_data = json.load(open(filename, encoding='utf-8'))

In [55]:
def get_text_location(geo_str):
    gu_dict = {}
    for gu in geo_str['features']:
        for coord in gu['geometry']['coordinates']:
            geo = np.array(coord)
            gu_dict[gu['id']] = [np.mean(geo[:,1]), np.mean(geo[:,0])]
    return gu_dict

In [56]:
map1 = folium.Map([37.55, 126.98], zoom_start=11, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_data,
    data=s.면적,
    columns=[s.index, s.면적],
    fill_color='YlOrRd',
    key_on='properties.name'
).add_to(map1)
gu_dict = get_text_location(geo_data)
for gu_name in s.index:
    folium.map.Marker(
        location=gu_dict[gu_name],
        icon = folium.DivIcon(icon_size=(80,20), icon_anchor=(20,0),
            html=f'<div style="font-size: 10pt">{gu_name}</div>'
        )
    ).add_to(map1)
map1

- 자치구별 인당 공원면적 단계 구분도

In [57]:
s_p = pd.read_csv('../02.Pandas/data/서울시인구.tsv', sep='\s+', skiprows=2)
s_p.head()

,기간,자치구,세대,계,남자,여자,계.1,남자.1,여자.1,계.2,남자.2,여자.2,세대당인구,65세이상고령자
0,2021,합계,"4,426,007","9,736,027","4,721,977","5,014,050","9,509,458","4,618,040","4,891,418","226,569","103,937","122,632",2.15,"1,605,416"
1,2021,종로구,"73,494","153,789","74,186","79,603","144,683","70,183","74,500","9,106","4,003","5,103",1.97,"27,818"
2,2021,중구,"63,519","131,787","64,083","67,704","122,499","59,630","62,869","9,288","4,453","4,835",1.93,"24,392"
3,2021,용산구,"111,036","237,285","115,085","122,200","222,953","107,210","115,743","14,332","7,875","6,457",2.01,"39,070"
4,2021,성동구,"134,233","292,672","142,259","150,413","285,990","139,380","146,610","6,682","2,879","3,803",2.13,"46,380"


In [58]:
s_p = s_p[['자치구', '계']]
s_p.drop([0], inplace=True)
columns = '자치구,계'.split(',')
for col in columns[1:]:
    s_p[col] = s_p[col].str.replace(',','').astype(int)
s_p.columns = ['주소','인구수']
s_p.sort_values('주소', inplace=True)
s_p.set_index('주소', inplace=True)
s_p.reset_index(inplace=True)
s_p.head()

,주소,인구수
0,강남구,537800
1,강동구,466472
2,강북구,302563
3,강서구,579768
4,관악구,499449


In [59]:
s.reset_index(inplace=True)

In [60]:
a = pd.merge(s, s_p)
a['인당공원면적'] = (a.면적 / a.인구수).round(3)
a.set_index('주소', inplace=True)
a.head()

,면적,인구수,인당공원면적
주소,,,
강남구,3626398,537800,6.743
강동구,1526387,466472,3.272
강북구,2047054,302563,6.766
강서구,2194152,579768,3.785
관악구,10431453,499449,20.886


In [61]:
map2 = folium.Map([37.55, 126.98], zoom_start=11, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_data,
    data=a.인당공원면적,
    columns=[a.index, a.인당공원면적],
    fill_color='YlOrRd',
    key_on='properties.name'
).add_to(map2)
gu_dict = get_text_location(geo_data)
for gu_name in a.index:
    folium.map.Marker(
        location=gu_dict[gu_name],
        icon = folium.DivIcon(icon_size=(80,20), icon_anchor=(20,0),
            html=f'<div style="font-size: 10pt">{gu_name}</div>'
        )
    ).add_to(map2)
map2

### 경기도 공원
- 시군구별 공원면적 단계 구분도

In [62]:
g_park = pd.read_csv('data/전국도시공원정보표준데이터.csv', encoding='euc-kr')
g_park.head(3)

,관리번호,공원명,공원구분,소재지도로명주소,소재지지번주소,위도,경도,공원면적,공원보유시설(운동시설),공원보유시설(유희시설),공원보유시설(편익시설),공원보유시설(교양시설),공원보유시설(기타시설),지정고시일,관리기관명,전화번호,데이터기준일자,제공기관코드,제공기관명
0,11170-00048,녹색소공원,소공원,NaN,서울특별시 용산구 효창동 287-6,37.539217,126.962694,1569.9,NaN,NaN,NaN,NaN,NaN,2012-10-19,서울특별시 용산구청,02-2199-7605,2022-12-20,3020000,서울특별시 용산구
1,36110-00058,푸름샘 소공원(소1-1),소공원,세종특별자치시 마음안로 161,NaN,36.516117,127.233781,1820.0,NaN,NaN,앉음벽,NaN,실개천,2011-08-05,세종특별자치시 공원관리사업소,044-301-3613,2022-12-14,5690000,세종특별자치시
2,36110-00059,솔숲 소공원(소1-2),소공원,세종특별자치시 만남1북로 73,NaN,36.508995,127.230040,1037.0,NaN,조합놀이대+ 그네+ 시소+ 흔들놀이,의자+ 파고라,NaN,안내판,2011-08-05,세종특별자치시 공원관리사업소,044-301-3613,2022-12-14,5690000,세종특별자치시


In [63]:
g_park = g_park[g_park['제공기관명'].str.contains('경기도')]
g_park = g_park[['소재지지번주소', '공원면적']]
g_park.dropna(subset=['소재지지번주소'], inplace=True)
gu_list = []
city = ['수원시', '고양시', '용인시', '성남시', '안산시', '안양시']
for line in g_park.소재지지번주소:
    if line.split()[1] in city:
        gu_line = line.split()[1:3]
        si = ''.join(gu_line)
    else:
        si = line.split()[1]
    gu_list.append(si)
g_park['소재지지번주소'] = gu_list
g_park = g_park.drop(7142)
g_park = g_park.drop(g_park[g_park.소재지지번주소.apply(lambda x: len(x) > 7)].index)
g_park = g_park.groupby('소재지지번주소')['공원면적'].sum()
g = pd.DataFrame(g_park)
g.head()

,공원면적
소재지지번주소,
가평군,511674.5
고양시덕양구,3287291.4
고양시일산동구,2722980.0
고양시일산서구,922486.9
과천시,315666.0


In [64]:
filename = 'data/경기도_geo_simple.json'
geo_data = json.load(open(filename, encoding='utf-8'))

In [65]:
map3 = folium.Map([37.4, 127.1], zoom_start=9, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_data,
    data=g.공원면적,
    columns=[g.index, g.공원면적],
    fill_color='YlOrRd',
    key_on='properties.name'
).add_to(map3)
def get_text_location(geo_str):
    gu_dict = {}
    for gu in geo_str['features']:
        for coord in gu['geometry']['coordinates']:
            geo = np.array(coord)
            gu_dict[gu['properties']['name']] = [np.mean(geo[:,1]), np.mean(geo[:,0])]
    return gu_dict
gu_dict = get_text_location(geo_data)
for gu_name in g.index:
    folium.map.Marker(
        location=gu_dict[gu_name],
        icon = folium.DivIcon(icon_size=(80,20), icon_anchor=(20,0),
            html=f'<div style="font-size: 10pt">{gu_name}</div>'
        )
    ).add_to(map3)
map3

- 시군구별 인당 공원면적 단계 구분도

In [66]:
g.reset_index(inplace=True)
g.head()

,소재지지번주소,공원면적
0,가평군,511674.5
1,고양시덕양구,3287291.4
2,고양시일산동구,2722980.0
3,고양시일산서구,922486.9
4,과천시,315666.0


In [67]:
d_p = pd.read_csv('../02.Pandas/data/주민등록인구집계현황.csv', encoding='euc-kr')
d_p.head(3)

,연도,월,행정구역구분명,행정구역명,총 인구수,0~9세,10~19세,20~29세,30~39세,40~49세,...,10~19세 (여),20~29세 (여),30~39세 (여),40~49세 (여),50~59세 (여),60~69세 (여),70~79세 (여),80~89세 (여),90~99세 (여),100세 이상 (여)
0,2022,12,도,경기도,13589432,1040954,1327726,1737291,1882300,2284603,...,647283,821150,904073,1121889,1133428,878494,437251,252711,44184,1369
1,2022,12,시군,경기도 가평군,62197,2930,4463,5929,5397,7589,...,2186,2503,2433,3525,5090,6439,3653,2408,411,13
2,2022,12,읍면동,경기도 가평군 가평읍,19555,1020,1673,2094,1795,2630,...,778,937,848,1244,1583,1805,1054,739,126,6


In [68]:
d_p.drop([0], inplace=True)
d_p = d_p[['행정구역명', '총 인구수']]
gu_list = []
city = ['수원시', '고양시', '용인시', '성남시', '안산시', '안양시']
for line in d_p.행정구역명:
    if line.split()[1] in city:
        gu_line = line.split()[1:3]
        si = ''.join(gu_line)
    else:
        si = line.split()[1]
    gu_list.append(si)
d_p['행정구역명'] = gu_list
d_p.columns = ['소재지지번주소','인구수']
d_p = d_p.groupby('소재지지번주소')['인구수'].sum()
d = pd.DataFrame(d_p)
d.reset_index(inplace=True)
d.head()

,소재지지번주소,인구수
0,가평군,124394
1,고양시,1075202
2,고양시덕양구,975748
3,고양시일산동구,593180
4,고양시일산서구,581476


In [69]:
b = pd.merge(g, d)
b['인당공원면적'] = (b.공원면적 / b.인구수).round(3)
b.set_index('소재지지번주소', inplace=True)
b.head()

,공원면적,인구수,인당공원면적
소재지지번주소,,,
가평군,511674.5,124394,4.113
고양시덕양구,3287291.4,975748,3.369
고양시일산동구,2722980.0,593180,4.590
고양시일산서구,922486.9,581476,1.586
과천시,315666.0,155550,2.029


In [70]:
map4 = folium.Map([37.4, 127.1], zoom_start=9, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_data,
    data=b.인당공원면적,
    columns=[b.index, b.인당공원면적],
    fill_color='YlOrRd',
    key_on='properties.name'
).add_to(map4)
def get_text_location(geo_str):
    gu_dict = {}
    for gu in geo_str['features']:
        for coord in gu['geometry']['coordinates']:
            geo = np.array(coord)
            gu_dict[gu['properties']['name']] = [np.mean(geo[:,1]), np.mean(geo[:,0])]
    return gu_dict
gu_dict = get_text_location(geo_data)
for gu_name in b.index:
    folium.map.Marker(
        location=gu_dict[gu_name],
        icon = folium.DivIcon(icon_size=(80,20), icon_anchor=(20,0),
            html=f'<div style="font-size: 10pt">{gu_name}</div>'
        )
    ).add_to(map4)
map4